In [1]:
# Paso 1: Carga de Datos
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [2]:
# Paso 2: Split de datos
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(train, test_size=0.2, random_state=42)

In [3]:
# Paso 3: EDA y Profiling de los datos
from pandas_profiling import ProfileReport


C:\Users\gehov\AppData\Local\Temp\ipykernel_11516\91327901.py:2: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [4]:
# Paso 4: Descripción de los warnings generados en el EDA
# Analizar los resultados del profiling y abordar cualquier problema identificado.

# Verificar la presencia de valores nulos
missing_values = train_data.isnull().sum()
print("Valores nulos por columna:")
print(missing_values[missing_values > 0])

# Verificar duplicados
duplicates = train_data.duplicated()
print("\nNúmero de duplicados:", duplicates.sum())

# Otras comprobaciones según tus necesidades

# ...


Valores nulos por columna:
LotFrontage      217
Alley           1094
MasVnrType         6
MasVnrArea         6
BsmtQual          28
BsmtCond          28
BsmtExposure      28
BsmtFinType1      28
BsmtFinType2      28
Electrical         1
FireplaceQu      547
GarageType        64
GarageYrBlt       64
GarageFinish      64
GarageQual        64
GarageCond        64
PoolQC          1162
Fence            935
MiscFeature     1122
dtype: int64

Número de duplicados: 0


In [5]:
# Paso 5: Setup - Configuración de PyCaret
from pycaret.regression import *

reg = setup(data=train_data, 
            target='SalePrice',
            numeric_imputation='mean',
            categorical_features=['MSZoning','Exterior1st','Exterior2nd','KitchenQual','Functional','SaleType',
                                  'Street','LotShape','LandContour','LotConfig','LandSlope','Neighborhood',   
                                  'Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',    
                                  'MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond',   
                                  'BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir',   
                                  'Electrical','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive',
                                  'SaleCondition'],
            ignore_features=['Alley','PoolQC','MiscFeature','Fence','FireplaceQu','Utilities'],
            feature_selection=True,
            pca=True,
            remove_multicollinearity=True,
            remove_outliers=True,
            normalize=True,
            verbose=True)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010118 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 64704
[LightGBM] [Info] Number of data points in the train set: 776, number of used features: 254
[LightGBM] [Info] Start training from score 178918.916237
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

,Description,Value
0,Session id,5693
1,Target,SalePrice
2,Target type,Regression
3,Original data shape,"(1168, 81)"
4,Transformed data shape,"(1127, 15)"
5,Transformed train set shape,"(776, 15)"
6,Transformed test set shape,"(351, 15)"
7,Ignore features,6
8,Ordinal features,2
9,Numeric features,37


In [6]:
# Paso 6: Entrenamiento y selección de modelos automática
compare_models(sort='RMSE') 


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,19924.9358,1026775756.7410,31144.8618,0.8024,0.1601,0.1174,9.4150
et,Extra Trees Regressor,19173.3072,1092484125.2382,31433.4899,0.7886,0.1558,0.1125,9.0320
rf,Random Forest Regressor,19941.8557,1100484648.6908,31796.7378,0.7875,0.1594,0.1166,11.2870
gbr,Gradient Boosting Regressor,19794.3268,1150286849.9034,32278.8840,0.7746,0.1592,0.1166,8.9690
en,Elastic Net,21202.3963,1166881756.4183,32771.7129,0.7735,0.1667,0.1249,6.4750
par,Passive Aggressive Regressor,21055.0610,1166643662.5866,33017.0850,0.7765,0.1637,0.1206,6.8170
huber,Huber Regressor,21206.8432,1184903959.8635,33244.6485,0.7715,0.1905,0.1233,6.3190
br,Bayesian Ridge,21706.2023,1218124029.3874,33311.9426,0.7605,0.1817,0.1298,6.3710
ridge,Ridge Regression,21865.7298,1248868541.2361,33748.0716,0.7543,0.1818,0.1313,6.1870
llar,Lasso Least Angle Regression,21879.6189,1252467370.7330,33798.9916,0.7535,0.1814,0.1314,6.3410


LGBMRegressor(n_jobs=-1, random_state=5693)

In [7]:
# Seleccionar los tres mejores modelos
top3_models = compare_models(n_select=3, sort='RMSE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,19924.9358,1026775756.7410,31144.8618,0.8024,0.1601,0.1174,10.9290
et,Extra Trees Regressor,19173.3072,1092484125.2382,31433.4899,0.7886,0.1558,0.1125,9.6080
rf,Random Forest Regressor,19941.8557,1100484648.6908,31796.7378,0.7875,0.1594,0.1166,10.2610
gbr,Gradient Boosting Regressor,19794.3268,1150286849.9034,32278.8840,0.7746,0.1592,0.1166,9.5110
en,Elastic Net,21202.3963,1166881756.4183,32771.7129,0.7735,0.1667,0.1249,6.5670
par,Passive Aggressive Regressor,21055.0610,1166643662.5866,33017.0850,0.7765,0.1637,0.1206,6.4820
huber,Huber Regressor,21206.8432,1184903959.8635,33244.6485,0.7715,0.1905,0.1233,6.6260
br,Bayesian Ridge,21706.2023,1218124029.3874,33311.9426,0.7605,0.1817,0.1298,7.0610
ridge,Ridge Regression,21865.7298,1248868541.2361,33748.0716,0.7543,0.1818,0.1313,6.4270
llar,Lasso Least Angle Regression,21879.6189,1252467370.7330,33798.9916,0.7535,0.1814,0.1314,6.5450


In [8]:
top3_models

[LGBMRegressor(n_jobs=-1, random_state=5693),
 ExtraTreesRegressor(n_jobs=-1, random_state=5693),
 RandomForestRegressor(n_jobs=-1, random_state=5693)]

In [28]:
# Selecciona los tres mejores modelos y entrena
top3_models = compare_models(n_select=3, sort='RMSE', verbose=False)

# Muestra la gráfica de importancia de características para cada modelo
for model in top3_models:
    plot_model(model, plot='feature', save=True)



In [9]:
# Paso 7: Optimización de hiper-parámetros
tuned_models = [tune_model(model) for model in top3_models]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,27073.6879,2661009593.6754,51584.9745,0.4840,0.2058,0.1493
1,19900.6932,856819312.1323,29271.4761,0.8650,0.1492,0.1139
2,21441.1550,1671511620.5250,40884.1243,0.5734,0.1761,0.1169
3,15631.9502,383360138.4210,19579.5847,0.8872,0.1240,0.1003
4,25129.0285,1736596429.2737,41672.4901,0.8009,0.1919,0.1402
5,17915.0344,573090775.1672,23939.3144,0.8529,0.1407,0.1100
6,21188.4697,800723818.3404,28297.0638,0.8596,0.1818,0.1403
7,19922.0374,877563430.2729,29623.6971,0.8176,0.1496,0.1142
8,23067.2091,910103966.1020,30167.9294,0.8207,0.1878,0.1445


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,25890.1559,2247410178.2780,47406.8579,0.5642,0.2004,0.1412
1,18884.5677,963088707.7442,31033.6705,0.8482,0.1605,0.1095
2,20538.1219,1558996672.6358,39484.1319,0.6021,0.1679,0.1128
3,17418.3500,579026130.2411,24062.9618,0.8296,0.1707,0.1221
4,23977.2385,1875235027.8911,43303.9840,0.7850,0.1828,0.1228
5,15333.8150,428792404.5091,20707.3032,0.8900,0.1161,0.0917
6,19313.8120,634703836.5781,25193.3292,0.8887,0.1582,0.1242
7,16913.0836,705161398.1353,26554.8752,0.8534,0.1336,0.0988
8,19941.0687,836254136.7999,28918.0590,0.8353,0.1616,0.1183


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,25928.8693,2420275420.6507,49196.2948,0.5307,0.1996,0.1401
1,18555.1418,789243633.1925,28093.4803,0.8756,0.1501,0.1084
2,20814.3418,1752047763.4211,41857.4696,0.5528,0.1721,0.1136
3,17156.8898,543972606.1790,23323.2203,0.8399,0.1595,0.1150
4,22930.1561,1589750999.7993,39871.6817,0.8178,0.1723,0.1222
5,13855.5436,350957257.5910,18733.8532,0.9099,0.1090,0.0843
6,18490.9926,631963338.1845,25138.8810,0.8892,0.1540,0.1182
7,19520.3207,881021071.4324,29681.9991,0.8168,0.1483,0.1099
8,19050.7235,720346077.4912,26839.2637,0.8581,0.1567,0.1150


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [10]:
# Paso 8: Selección del modelo final
best_model = automl(optimize='rmse')

In [11]:
# Paso 9: Almacenamiento del pipeline en el disco duro
save_model(best_model, 'house_price_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Id', 'MSSubClass', 'LotFrontage',
                                              'LotArea', 'OverallQual',
                                              'OverallCond', 'YearBuilt',
                                              'YearRemodAdd', 'MasVnrArea',
                                              'BsmtFinSF1', 'BsmtFinSF2',
                                              'BsmtUnfSF', 'TotalBsmtSF',
                                              '1stFlrSF', '2ndFlrSF',
                                              'LowQualFinSF', 'GrLivArea',
                                              'BsmtFullBath', 'BsmtHalfBath',
                                              'FullBath', 'Hal...
                  TransformerWrapper(transformer=RemoveOutliers(random_state=5693))),
                 ('normalize', TransformerWrapper(transformer=StandardScaler())),
             

In [12]:
# Paso 10: Predicciones
loaded_model = load_model('house_price_model')
predictions = predict_model(loaded_model, data=test_data)

Transformation Pipeline and Model Successfully Loaded


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,21667.2219,1366975207.5560,36972.6278,0.8218,0.1813,0.1333


In [21]:
predictions

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,prediction_label
892,893,20,RL,70.0,8414,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,NaN,0,2,2006,WD,Normal,154500,136206.867118
1105,1106,60,RL,98.0,12256,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,4,2010,WD,Normal,325000,314469.136928
413,414,30,RM,56.0,8960,Pave,Grvl,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,3,2010,WD,Normal,115000,100676.612294
522,523,50,RM,50.0,5000,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,10,2006,WD,Normal,159000,140646.260870
1036,1037,20,RL,89.0,12898,Pave,NaN,IR1,HLS,AllPub,...,NaN,NaN,NaN,0,9,2009,WD,Normal,315500,323154.453005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,480,30,RM,50.0,5925,Pave,NaN,Reg,Bnk,AllPub,...,NaN,MnPrv,NaN,0,3,2007,WD,Alloca,89471,116587.127594
1361,1362,20,RL,124.0,16158,Pave,NaN,IR1,Low,AllPub,...,NaN,NaN,NaN,0,6,2009,WD,Normal,260000,335110.019732
802,803,60,RL,63.0,8199,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,10,2008,WD,Normal,189000,221018.128964
651,652,70,RL,60.0,9084,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,NaN,0,10,2009,WD,Normal,108000,134104.698941


In [23]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Reemplaza 'SalePrice' con el nombre real de la columna de la variable objetivo
target_column = 'SalePrice'

# Reemplaza 'Label' con el nombre real de la columna de predicciones generada por PyCaret
predictions_column = 'prediction_label'

# Calcula las métricas de rendimiento
r2 = r2_score(test_data[target_column], predictions[predictions_column])
rmse = mean_squared_error(test_data[target_column], predictions[predictions_column], squared=False)
mse = mean_squared_error(test_data[target_column], predictions[predictions_column])
mape = mean_absolute_error(test_data[target_column], predictions[predictions_column]) / test_data[target_column].mean() * 100

# Muestra las métricas
print(f'R2: {r2}\nRMSE: {rmse}\nMSE: {mse}\nMAPE: {mape}%')


R2: 0.8217839540807041
RMSE: 36972.627815128835
MSE: 1366975207.5560386
MAPE: 12.115435413007685%


#El modelo tiene un buen ajuste y explica una proporción significativa de la variabilidad en los precios de las casas. Las predicciones, en términos de RMSE, tienen una desviación promedio de alrededor de 36,972 unidades. El MAPE del 12.12% sugiere que las predicciones tienen un error moderado en términos porcentuales. Estas métricas indican un rendimiento general sólido del modelo.





